In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

2024-02-26 17:13:20.854458: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 17:13:20.854497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 17:13:20.855925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-26 17:13:20.862465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 17:13:21.712357: W tensorflow/compiler/tf2

In [298]:
import numpy as np
import pandas as pd
from scipy.io import arff
data, meta = arff.loadarff('data/ECG5000_TEST.arff') # tor file er link dibi
df=pd.DataFrame(data)
df.shape

(4500, 141)

In [299]:
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,3.690844,0.711414,-2.114091,-4.141007,-4.574472,-3.431909,-1.950791,-1.107067,-0.632322,0.334577,...,0.022847,0.188937,0.480932,0.629250,0.577291,0.665527,1.035997,1.492287,-1.905073,b'1'
1,-1.348132,-3.996038,-4.226750,-4.251187,-3.477953,-2.228422,-1.808488,-1.534242,-0.779861,-0.397999,...,1.570938,1.591394,1.549193,1.193077,0.515134,0.126274,0.267532,1.071148,-1.164009,b'1'
2,1.024295,-0.590314,-1.916949,-2.806989,-3.527905,-3.638675,-2.779767,-2.019031,-1.980754,-1.440680,...,0.443502,0.827582,1.237007,1.235121,1.738103,1.800767,1.816301,1.473963,1.389767,b'1'
3,0.545657,-1.014383,-2.316698,-3.634040,-4.196857,-3.758093,-3.194444,-2.221764,-1.588554,-1.202146,...,0.777530,1.119240,0.902984,0.554098,0.497053,0.418116,0.703108,1.064602,-0.044853,b'1'
4,0.661133,-1.552471,-3.124641,-4.313351,-4.017042,-3.005993,-1.832411,-1.503886,-1.071705,-0.521316,...,1.280823,1.494315,1.618764,1.447449,1.238577,1.749692,1.986803,1.422756,-0.357784,b'1'


In [300]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objs as go
import plotly.express as px
from scipy.signal import medfilt, butter, filtfilt
import pywt
from sklearn.model_selection import train_test_split
import scipy.signal
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

In [301]:
df.columns

Index(['att1', 'att2', 'att3', 'att4', 'att5', 'att6', 'att7', 'att8', 'att9',
       'att10',
       ...
       'att132', 'att133', 'att134', 'att135', 'att136', 'att137', 'att138',
       'att139', 'att140', 'target'],
      dtype='object', length=141)

In [302]:
df['target']==b'1'

0        True
1        True
2        True
3        True
4        True
        ...  
4495    False
4496    False
4497    False
4498    False
4499    False
Name: target, Length: 4500, dtype: bool

In [303]:
df.shape

(4500, 141)

In [304]:
c12 = df[df.iloc[:,140] ==b'1']
c12.shape

(2627, 141)

In [305]:
#plot graphs of normal and abnormal ECG to visualise the trends
c1 = df[df.iloc[:,140] ==b'1'][:10]
c2 = df[df.iloc[:,140] ==b'2'][:10]
c3 = df[df.iloc[:,140] ==b'3'][:10]
c4 = df[df.iloc[:,140] ==b'4'][:10]
c5 = df[df.iloc[:,140] ==b'5'][:10]
# Create the figure
fig = go.Figure()
#create a list to display only a single legend
leg1  = [False] * c1.shape[0]
leg1[0] = True

leg2  = [False] * c2.shape[0]
leg2[0] = True

leg3  = [False] * c3.shape[0]
leg3[0] = True

leg4  = [False] * c4.shape[0]
leg4[0] = True

leg5  = [False] * c5.shape[0]
leg5[0] = True

c1

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,3.690844,0.711414,-2.114091,-4.141007,-4.574472,-3.431909,-1.950791,-1.107067,-0.632322,0.334577,...,0.022847,0.188937,0.480932,0.629250,0.577291,0.665527,1.035997,1.492287,-1.905073,b'1'
1,-1.348132,-3.996038,-4.226750,-4.251187,-3.477953,-2.228422,-1.808488,-1.534242,-0.779861,-0.397999,...,1.570938,1.591394,1.549193,1.193077,0.515134,0.126274,0.267532,1.071148,-1.164009,b'1'
2,1.024295,-0.590314,-1.916949,-2.806989,-3.527905,-3.638675,-2.779767,-2.019031,-1.980754,-1.440680,...,0.443502,0.827582,1.237007,1.235121,1.738103,1.800767,1.816301,1.473963,1.389767,b'1'
3,0.545657,-1.014383,-2.316698,-3.634040,-4.196857,-3.758093,-3.194444,-2.221764,-1.588554,-1.202146,...,0.777530,1.119240,0.902984,0.554098,0.497053,0.418116,0.703108,1.064602,-0.044853,b'1'
4,0.661133,-1.552471,-3.124641,-4.313351,-4.017042,-3.005993,-1.832411,-1.503886,-1.071705,-0.521316,...,1.280823,1.494315,1.618764,1.447449,1.238577,1.749692,1.986803,1.422756,-0.357784,b'1'
5,1.649272,1.121169,-0.628753,-1.939899,-3.439175,-4.010777,-3.238067,-2.127737,-1.798464,-1.323561,...,0.636727,1.166689,1.062871,1.628234,2.175072,2.500749,2.092176,1.455313,2.124303,b'1'
6,-0.323103,-1.510804,-2.768216,-3.788649,-4.100396,-3.516576,-2.940032,-1.828790,-1.089045,-0.858473,...,1.052238,0.213416,-1.003917,-1.676931,-2.055868,-1.968963,-1.548612,-1.411230,-2.620855,b'1'
7,-1.309274,-4.045362,-4.757328,-4.657040,-3.634479,-2.469447,-1.640088,-1.408992,-0.706937,-0.272103,...,1.371353,1.257078,1.246963,1.156629,0.311326,-0.325923,-0.385273,0.385835,-0.393619,b'1'
8,-1.736906,-3.647089,-4.005470,-4.413745,-4.082557,-3.234594,-1.935797,-1.367653,-1.055027,-0.545169,...,1.130086,1.317606,1.181094,0.976701,0.541003,0.288060,0.479704,0.658353,-0.326727,b'1'
9,-1.101873,-1.965967,-2.399643,-2.905194,-3.107600,-3.041231,-2.105358,-1.621213,-1.589307,-1.232691,...,1.436628,0.931638,0.332091,0.409455,0.489593,0.890026,1.026954,0.937848,0.321598,b'1'


In [306]:
import plotly.graph_objects as go
import numpy as np

# Assuming df is your DataFrame

c1 = df[df.iloc[:, 140] == b'1'][:10]
c2 = df[df.iloc[:, 140] == b'2'][:10]
c3 = df[df.iloc[:, 140] == b'3'][:10]
c4 = df[df.iloc[:, 140] == b'4'][:10]
c5 = df[df.iloc[:, 140] == b'5'][:10]

# Convert bytes to numeric values
c1 = c1.apply(lambda x: pd.to_numeric(x, errors='coerce'))
c2 = c2.apply(lambda x: pd.to_numeric(x, errors='coerce'))
c3 = c3.apply(lambda x: pd.to_numeric(x, errors='coerce'))
c4 = c4.apply(lambda x: pd.to_numeric(x, errors='coerce'))
c5 = c5.apply(lambda x: pd.to_numeric(x, errors='coerce'))

# Create the figure
fig = go.Figure()

# Create a list to display only a single legend
leg1 = [False] * c1.shape[0]
leg1[0] = True

leg2 = [False] * c2.shape[0]
leg2[0] = True

leg3 = [False] * c3.shape[0]
leg3[0] = True

leg4 = [False] * c4.shape[0]
leg4[0] = True

leg5 = [False] * c5.shape[0]
leg5[0] = True

# Plot abnormal ECG
for i in range(c1.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(c1.shape[1]), y=c1.iloc[i, :], name='c1', mode='lines',
                             marker_color='rgba(255, 0, 0, 0.9)', showlegend=leg1[i]))

for i in range(c2.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(c2.shape[1]), y=c2.iloc[i, :], name='c2', mode='lines',
                             marker_color='rgba(0, 255, 0, 0.9)', showlegend=leg2[i]))

for i in range(c3.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(c3.shape[1]), y=c3.iloc[i, :], name='c3', mode='lines',
                             marker_color='rgba(0, 0, 255, 0.9)', showlegend=leg3[i]))

for i in range(c4.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(c4.shape[1]), y=c4.iloc[i, :], name='c4', mode='lines',
                             marker_color='rgba(255, 255, 0, 0.9)', showlegend=leg4[i]))

for i in range(c5.shape[0]):
    fig.add_trace(go.Scatter(x=np.arange(c5.shape[1]), y=c5.iloc[i, :], name='c5', mode='lines',
                             marker_color='rgba(255, 0, 255, 0.9)', showlegend=leg5[i]))

# Update layout
fig.update_layout(xaxis_title="time", yaxis_title="Signal",
                  title={'text': 'Difference between different ECG', 'xanchor': 'center', 'yanchor': 'top', 'x': 0.5},
                  bargap=0,)
fig.update_traces(opacity=0.5)
fig.show()


In [11]:
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,3.690844,0.711414,-2.114091,-4.141007,-4.574472,-3.431909,-1.950791,-1.107067,-0.632322,0.334577,...,0.022847,0.188937,0.480932,0.629250,0.577291,0.665527,1.035997,1.492287,-1.905073,b'1'
1,-1.348132,-3.996038,-4.226750,-4.251187,-3.477953,-2.228422,-1.808488,-1.534242,-0.779861,-0.397999,...,1.570938,1.591394,1.549193,1.193077,0.515134,0.126274,0.267532,1.071148,-1.164009,b'1'
2,1.024295,-0.590314,-1.916949,-2.806989,-3.527905,-3.638675,-2.779767,-2.019031,-1.980754,-1.440680,...,0.443502,0.827582,1.237007,1.235121,1.738103,1.800767,1.816301,1.473963,1.389767,b'1'
3,0.545657,-1.014383,-2.316698,-3.634040,-4.196857,-3.758093,-3.194444,-2.221764,-1.588554,-1.202146,...,0.777530,1.119240,0.902984,0.554098,0.497053,0.418116,0.703108,1.064602,-0.044853,b'1'
4,0.661133,-1.552471,-3.124641,-4.313351,-4.017042,-3.005993,-1.832411,-1.503886,-1.071705,-0.521316,...,1.280823,1.494315,1.618764,1.447449,1.238577,1.749692,1.986803,1.422756,-0.357784,b'1'


split the data

In [307]:
ecg_data = df.iloc[:,:-1]
labels = df.iloc[:,-1]
labels.shape

(4500,)

In [308]:
labels=np.where(labels == b'1',1,0)

In [14]:
num_ones = np.count_nonzero(labels == 0)
num_ones

1873

In [309]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [310]:
labels_1 = labels[labels == 1]


In [311]:
labels_1

array([1, 1, 1, ..., 1, 1, 1])

# Take only normal 1 label data


In [312]:
ecg_data_label_0 = ecg_data[labels == 0]
ecg_data_label_1 = ecg_data[labels == 1]
ecg_data_label_1.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att131,att132,att133,att134,att135,att136,att137,att138,att139,att140
0,3.690844,0.711414,-2.114091,-4.141007,-4.574472,-3.431909,-1.950791,-1.107067,-0.632322,0.334577,...,-0.032477,0.022847,0.188937,0.480932,0.629250,0.577291,0.665527,1.035997,1.492287,-1.905073
1,-1.348132,-3.996038,-4.226750,-4.251187,-3.477953,-2.228422,-1.808488,-1.534242,-0.779861,-0.397999,...,0.937984,1.570938,1.591394,1.549193,1.193077,0.515134,0.126274,0.267532,1.071148,-1.164009
2,1.024295,-0.590314,-1.916949,-2.806989,-3.527905,-3.638675,-2.779767,-2.019031,-1.980754,-1.440680,...,0.368820,0.443502,0.827582,1.237007,1.235121,1.738103,1.800767,1.816301,1.473963,1.389767
3,0.545657,-1.014383,-2.316698,-3.634040,-4.196857,-3.758093,-3.194444,-2.221764,-1.588554,-1.202146,...,0.894638,0.777530,1.119240,0.902984,0.554098,0.497053,0.418116,0.703108,1.064602,-0.044853
4,0.661133,-1.552471,-3.124641,-4.313351,-4.017042,-3.005993,-1.832411,-1.503886,-1.071705,-0.521316,...,1.416582,1.280823,1.494315,1.618764,1.447449,1.238577,1.749692,1.986803,1.422756,-0.357784


In [313]:
ecg_data_label_1.shape

(2627, 140)

scale the data

In [314]:
scaler = MinMaxScaler(feature_range=(-1, 1))
ecg_data = scaler.fit_transform(ecg_data_label_1)

In [315]:
ecg_data.shape

(2627, 140)

filtering

In [316]:
#filtering the raw signals
# Median filtering
ecg_medfilt = medfilt(ecg_data, kernel_size=3)
# Low-pass filtering
lowcut = 0.05
highcut = 20.0
nyquist = 0.5 * 360.0
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')
ecg_lowpass = filtfilt(b, a, ecg_data)
# Wavelet filtering
coeffs = pywt.wavedec(ecg_data, 'db4', level=1)
threshold = np.std(coeffs[-1]) * np.sqrt(2*np.log(len(ecg_data)))
coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
ecg_wavelet = pywt.waverec(coeffs, 'db4')

filter data plots

In [317]:
# Plot original ECG signal
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(ecg_data.shape[0]), y=ecg_data[30], mode='lines', name='Original ECG signal'))
# Plot filtered ECG signals
fig.add_trace(go.Scatter(x=np.arange(ecg_medfilt.shape[0]), y=ecg_medfilt[30], mode='lines', name='Median filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(ecg_lowpass.shape[0]), y=ecg_lowpass[30], mode='lines', name='Low-pass filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(ecg_wavelet.shape[0]), y=ecg_wavelet[30], mode='lines', name='Wavelet filtered ECG signal'))
fig.show()

# taking wavelet filtered data

pad with 0

In [318]:
def pad_data(original_data,filtered_data):
  # Calculate the difference in length between the original data and filtered data
  diff = original_data.shape[1] - filtered_data.shape[1]
    # pad the shorter array with zeroes
  if diff > 0:
          # Create an array of zeros with the same shape as the original data
      padding = np.zeros((filtered_data.shape[0], original_data.shape[1]))
      # Concatenate the filtered data with the padding
      padded_data = np.concatenate((filtered_data, padding))
  elif diff < 0:
      padded_data = filtered_data[:,:-abs(diff)]
  elif diff == 0:
      padded_data = filtered_data
  return padded_data

In [319]:
def mse(original_data, filtered_data):
    filter_data = pad_data(original_data,filtered_data)
    return np.mean((original_data - filter_data) ** 2)
# Calculate MSE
mse_value_m = mse(ecg_data, ecg_medfilt)
mse_value_l = mse(ecg_data, ecg_lowpass)
mse_value_w = mse(ecg_data, ecg_wavelet)
print("MSE value of Median Filtering:", mse_value_m)
print("MSE value of Low-pass Filtering:", mse_value_l)
print("MSE value of Wavelet Filtering:", mse_value_w)

MSE value of Median Filtering: 0.035447259793798834
MSE value of Low-pass Filtering: 0.3277290161930179
MSE value of Wavelet Filtering: 0.0030034196549888737


# Train test split

In [356]:
X_train, X_test, y_train, y_test = train_test_split(ecg_wavelet, labels_1, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train[:10])

(2364, 140)
(263, 140)
[1 1 1 1 1 1 1 1 1 1]


# Feature extraction

In [392]:
# Initializing an empty list to store the features
features = []
# Extracting features for each sample
for i in range(X_train.shape[0]):
    #Finding the R-peaks
    r_peaks = scipy.signal.find_peaks(X_train[i])[0]
    #Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        t_peak = np.argmin(X_train[i][r_peak:r_peak+200]) + r_peak
        #Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_train[i][r_peak])
        t_amplitudes.append(X_train[i][t_peak])
    # extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    # extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = (len(X_train[i]) - 1) / 1000 # assuming data is in ms
    # Calculate the sampling rate
    sampling_rate = len(X_train[i]) / time_duration
    # Calculate heart rate
    duration = len(X_train[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60
    # QRS duration
    qrs_duration = []
    for j in range(len(r_peaks)):
        qrs_duration.append(r_peaks[j]-r_peaks[j-1])
    # extracting singular value metrics from the qrs_durations
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the singular value metrics from the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
    # Extracting the overall standard deviation
    std = np.std(X_train[i])
    # Extracting the overall mean
    mean = np.mean(X_train[i])
    # Appending the features to the list
    features.append([mean, std, std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr, heart_rate])
# Converting the list to a numpy array
features = np.array(features)

In [27]:
features.shape

(2364, 19)

# Binary class


In [28]:
# y_train=np.where(y_train == b'1', 1, 0)
# np.unique(y_train)
y_train

array([1, 1, 1, ..., 1, 1, 1])

In [29]:
# y_test=np.where(y_test == b'1', 1, 0)
# np.unique(y_train)
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

test feature extraction

In [394]:
# Initializing an empty list to store the features
X_test_fe = []
# Extracting features for each sample
for i in range(X_test.shape[0]):
    # Finding the R-peaks
    r_peaks = scipy.signal.find_peaks(X_test[i])[0]
    # Initialize lists to hold R-peak and T-peak amplitudes
    r_amplitudes = []
    t_amplitudes = []
    # Iterate through R-peak locations to find corresponding T-peak amplitudes
    for r_peak in r_peaks:
        # Find the index of the T-peak (minimum value) in the interval from R-peak to R-peak + 200 samples
        t_peak = np.argmin(X_test[i][r_peak:r_peak+200]) + r_peak
        # Append the R-peak amplitude and T-peak amplitude to the lists
        r_amplitudes.append(X_test[i][r_peak])
        t_amplitudes.append(X_test[i][t_peak])
    #extracting singular value metrics from the r_amplitudes
    std_r_amp = np.std(r_amplitudes)
    mean_r_amp = np.mean(r_amplitudes)
    median_r_amp = np.median(r_amplitudes)
    sum_r_amp = np.sum(r_amplitudes)
    #extracting singular value metrics from the t_amplitudes
    std_t_amp = np.std(t_amplitudes)
    mean_t_amp = np.mean(t_amplitudes)
    median_t_amp = np.median(t_amplitudes)
    sum_t_amp = np.sum(t_amplitudes)
    # Find the time between consecutive R-peaks
    rr_intervals = np.diff(r_peaks)
    # Calculate the time duration of the data collection
    time_duration = (len(X_test[i]) - 1) / 1000 # assuming data is in ms
    # Calculate the sampling rate
    sampling_rate = len(X_test[i]) / time_duration
    # Calculate heart rate
    duration = len(X_test[i]) / sampling_rate
    heart_rate = (len(r_peaks) / duration) * 60
    # QRS duration
    qrs_duration = []
    for j in range(len(r_peaks)):
        qrs_duration.append(r_peaks[j]-r_peaks[j-1])
    #extracting singular value metrics from the qrs_duartions
    std_qrs = np.std(qrs_duration)
    mean_qrs = np.mean(qrs_duration)
    median_qrs = np.median(qrs_duration)
    sum_qrs = np.sum(qrs_duration)
    # Extracting the standard deviation of the RR-interval
    std_rr = np.std(rr_intervals)
    mean_rr = np.mean(rr_intervals)
    median_rr = np.median(rr_intervals)
    sum_rr = np.sum(rr_intervals)
      # Extracting the standard deviation of the RR-interval
    std = np.std(X_test[i])
    # Extracting the mean of the RR-interval
    mean = np.mean(X_test[i])
    # Appending the features to the list
    X_test_fe.append([mean, std,  std_qrs, mean_qrs,median_qrs, sum_qrs, std_r_amp, mean_r_amp, median_r_amp, sum_r_amp, std_t_amp, mean_t_amp, median_t_amp, sum_t_amp, sum_rr, std_rr, mean_rr,median_rr,heart_rate])
# Converting the list to a numpy array
X_test_fe = np.array(X_test_fe)

In [395]:
X_test_fe.shape

(263, 19)

# Transformer Model

In [396]:
X_train.shape

(2364, 140)

In [33]:
X_test.shape

(263, 140)

In [34]:
# features.shape
y_test.shape

(263,)

In [35]:
# features[0]
y_train.shape

(2364,)

In [36]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [397]:
array_flattened = features.flatten().reshape(-1, 1)
array_flattened.shape

(44916, 1)

In [398]:
array_flattened_test = X_test_fe.flatten().reshape(-1, 1)
array_flattened_test.shape

(4997, 1)

In [440]:
array_flattened = X_train.flatten().reshape(-1, 1)
array_flattened.shape

(330960, 1)

In [441]:
array_flattened_test = X_test.flatten().reshape(-1, 1)
array_flattened_test.shape

(36820, 1)

In [442]:
# updated sequence
SEQUENCE_SIZE = 10
# SEQUENCE_SIZE = 9

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

x_train, y_s_train = to_sequences(SEQUENCE_SIZE, array_flattened)
x_test, y_s_test = to_sequences(SEQUENCE_SIZE, array_flattened_test)

# Setup data loaders for batch
# train_dataset = TensorDataset(x_train, y_train)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

# test_dataset = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [443]:
# x_train[0]
# y_s_train.shape
print(x_train.shape)
print(x_test.shape)

torch.Size([330950, 10, 1])
torch.Size([36810, 10, 1])


In [428]:
y_s_test

tensor([[ 1.8470e+00],
        [ 2.4518e-01],
        [-4.8471e-01],
        ...,
        [ 6.0476e+00],
        [ 5.0000e+00],
        [ 9.4964e+03]])

In [512]:
# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_s_train)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = TensorDataset(x_test, y_s_test)
test_loader = DataLoader(test_dataset, batch_size=140, shuffle=False)

In [445]:
valid_dataset = TensorDataset(x_test, y_s_test)
valid_loader = DataLoader(valid_dataset, batch_size=140, shuffle=False)

In [446]:
train_dataset_thr = TensorDataset(x_train, y_s_train)
train_loader_thr = DataLoader(train_dataset, batch_size=140, shuffle=False)

In [435]:
sample_features, sample_label = train_dataset[0]
print(sample_features)
print(sample_label)

tensor([[2.3501e-02],
        [2.8373e-01],
        [2.8042e+01],
        [0.0000e+00],
        [5.0000e+00],
        [0.0000e+00],
        [3.0079e-01],
        [1.3226e-01],
        [1.8638e-01]])
tensor([3.0421])


In [513]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: not using Google CoLab
Using device: cuda


In [597]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)



class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=256, nhead=32, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])

        return x

T_model = TransformerModel().to(device)

In [598]:
# Train the model
# 
# criterion = nn.L1Loss()
criterion = nn.MSELoss()

optimizer = torch.optim.Adam(T_model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 1000
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):
    T_model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = T_model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    # Validation
    T_model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = T_model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)

    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 5:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {val_loss:.4f}")

Epoch 1/1000, Validation Loss: 0.0156
Epoch 2/1000, Validation Loss: 0.0168
Epoch 3/1000, Validation Loss: 0.0150
Epoch 4/1000, Validation Loss: 0.0158
Epoch 5/1000, Validation Loss: 0.0153
Epoch 6/1000, Validation Loss: 0.0157
Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.
Epoch 7/1000, Validation Loss: 0.0154
Early stopping!


In [142]:
T_model.eval()
val_losses = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = T_model(x_batch)
        loss = criterion(outputs, y_batch)
        val_losses.append(loss.item())

In [52]:
len(val_losses)

1151

In [143]:
val_thresh=np.mean(val_losses) + np.std(val_losses)
val_thresh

0.0043123092324549445

In [599]:
criterion = nn.L1Loss()
T_model.eval()
train_losses = []
with torch.no_grad():
    for batch in train_loader_thr:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = T_model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())

In [600]:
train_thresh=np.mean(train_losses) +  np.std(train_losses)
train_thresh

0.0948076786638575

In [487]:
criterion = nn.L1Loss()
T_model.eval()
val_losses = []
count=0
with torch.no_grad():
    for batch in valid_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = T_model(x_batch)
        loss = criterion(outputs, y_batch)
        if (loss.item() > train_thresh):
          count +=1
        val_losses.append(loss.item())
count

263

In [456]:
val_thresh=np.mean(val_losses) + np.std(val_losses)
val_thresh

0.024803898634682034

In [58]:
len(val_losses)

263

# Test Data

In [376]:
import numpy as np
import pandas as pd
from scipy.io import arff
test_data, test_meta = arff.loadarff('data/ECG5000_TRAIN.arff') # tor file er link dibi
test_df=pd.DataFrame(test_data)
test_df.shape

(500, 141)

In [377]:
test_ecg_data = test_df.iloc[:,:-1]
test_labels = test_df.iloc[:,-1]
test_labels.shape

(500,)

In [378]:
test_labels=np.where(test_labels == b'1',0,1)

In [379]:
scaler = MinMaxScaler(feature_range=(-1, 1))
test_ecg_data = scaler.fit_transform(test_ecg_data)

In [380]:
test_ecg_data.shape

(500, 140)

In [381]:
#filtering the raw signals
# Median filtering
test_ecg_medfilt = medfilt(test_ecg_data, kernel_size=3)
# Low-pass filtering
lowcut = 0.05
highcut = 20.0
nyquist = 0.5 * 360.0
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(4, [low, high], btype='band')
test_ecg_lowpass = filtfilt(b, a, test_ecg_data)
# Wavelet filtering
coeffs = pywt.wavedec(test_ecg_data, 'db4', level=1)
threshold = np.std(coeffs[-1]) * np.sqrt(2*np.log(len(test_ecg_data)))
coeffs[1:] = (pywt.threshold(i, value=threshold, mode='soft') for i in coeffs[1:])
test_ecg_wavelet = pywt.waverec(coeffs, 'db4')


# Plot original ECG signal
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(test_ecg_data.shape[0]), y=test_ecg_data[30], mode='lines', name='Original ECG signal'))
# Plot filtered ECG signals
fig.add_trace(go.Scatter(x=np.arange(test_ecg_medfilt.shape[0]), y=test_ecg_medfilt[30], mode='lines', name='Median filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(test_ecg_lowpass.shape[0]), y=test_ecg_lowpass[30], mode='lines', name='Low-pass filtered ECG signal'))
fig.add_trace(go.Scatter(x=np.arange(test_ecg_wavelet.shape[0]), y=test_ecg_wavelet[30], mode='lines', name='Wavelet filtered ECG signal'))
fig.show()

In [382]:
def pad_data(original_data,filtered_data):
  # Calculate the difference in length between the original data and filtered data
  diff = original_data.shape[1] - filtered_data.shape[1]
    # pad the shorter array with zeroes
  if diff > 0:
          # Create an array of zeros with the same shape as the original data
      padding = np.zeros((filtered_data.shape[0], original_data.shape[1]))
      # Concatenate the filtered data with the padding
      padded_data = np.concatenate((filtered_data, padding))
  elif diff < 0:
      padded_data = filtered_data[:,:-abs(diff)]
  elif diff == 0:
      padded_data = filtered_data
  return padded_data


def mse(original_data, filtered_data):
    filter_data = pad_data(original_data,filtered_data)
    return np.mean((original_data - filter_data) ** 2)
# Calculate MSE
mse_value_m = mse(test_ecg_data,  test_ecg_medfilt)
mse_value_l = mse(test_ecg_data, test_ecg_lowpass)
mse_value_w = mse(test_ecg_data, test_ecg_wavelet)
print("MSE value of Median Filtering:", mse_value_m)
print("MSE value of Low-pass Filtering:", mse_value_l)
print("MSE value of Wavelet Filtering:", mse_value_w)

MSE value of Median Filtering: 0.04334328419796525
MSE value of Low-pass Filtering: 0.5636105991314301
MSE value of Wavelet Filtering: 0.002151222541174613


In [526]:
test_array_flattened = test_ecg_data.flatten().reshape(-1, 1)
test_array_flattened.shape

(70000, 1)

In [527]:
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(len(obs) - seq_size):
        window = obs[i:(i + seq_size)]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32).view(-1, seq_size, 1), torch.tensor(y, dtype=torch.float32).view(-1, 1)

test_x, test_y_s = to_sequences(SEQUENCE_SIZE, test_array_flattened)


In [528]:
final_test_dataset = TensorDataset(test_x, test_y_s)
final_test_loader = DataLoader(final_test_dataset, batch_size=140, shuffle=False)

In [476]:
c=np.count_nonzero(test_labels==1)
c

208

In [603]:
criterion = nn.L1Loss()
T_model.eval()
val_losses = []
count=0
anomaly_indx=[]
with torch.no_grad():
    for batch in final_test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = T_model(x_batch)
        loss = criterion(outputs, y_batch)
        if (loss.item() > train_thresh):
          anomaly_indx.append(0)
          count += 1
        else:
          anomaly_indx.append(1)
        val_losses.append(loss.item())
count

327

In [604]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_labels, anomaly_indx)
all_met = classification_report(test_labels, anomaly_indx)
print("Accuracy: ", accuracy*100, "%")


print("Classification Report: \n", all_met)


Accuracy:  86.6 %
Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.95      0.89       292
           1       0.91      0.75      0.82       208

    accuracy                           0.87       500
   macro avg       0.88      0.85      0.86       500
weighted avg       0.87      0.87      0.86       500



# Lstm model

In [72]:
# Define the number of features in the train dataframe
num_features = features.shape[1]
# Reshape the features data to be in the right shape for LSTM input
features = np.asarray(features).astype('float32')
features = features.reshape(features.shape[0], features.shape[1], 1)
X_test_fe = X_test_fe.reshape(X_test_fe.shape[0], X_test_fe.shape[1], 1)
# Define the model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(features.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(features, y_train, validation_data=(X_test_fe, y_test), epochs=50, batch_size=32)
# Make predictions on the validation set
y_pred = model.predict(X_test_fe)
y_pred

2024-02-26 17:17:26.825009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-26 17:17:26.828610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-26 17:17:26.830761: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/50


2024-02-26 17:17:28.189548: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


 1/74 [..............................] - ETA: 1:39 - loss: 0.4654 - accuracy: 1.0000

2024-02-26 17:17:28.508124: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f832c8f47c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-26 17:17:28.508160: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-02-26 17:17:28.521595: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708946248.604767 3638728 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - 2s 7ms/step - loss: 0.1020 - accuracy: 1.0000 - val_loss: 0.0159 - val_accuracy: 1.0000
Epoch 2/50
74/74 [==============================] - 0s 4ms/step - loss: 0.0118 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 1.0000
Epoch 3/50
74/74 [==============================] - 0s 3ms/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.0058 - val_accuracy: 1.0000
Epoch 4/50
74/74 [==============================] - 0s 3ms/step - loss: 0.0049 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 5/50
74/74 [==============================] - 0s 4ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0032 - val_accuracy: 1.0000
Epoch 6/50
74/74 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 7/50
74/74 [==============================] - 0s 4ms/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 8/50
74/74 [=============

array([[0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999763],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],
       [0.9999764],


# without feature selection

In [73]:
ecg_data = df.iloc[:,:-1]
labels = df.iloc[:,-1]
labels.shape

(4500,)

In [74]:
labels=np.where(labels == b'1',1,0)
scaler = MinMaxScaler(feature_range=(-1, 1))
ecg_data_train = scaler.fit_transform(ecg_data)

In [75]:
ecg_data_train.shape

(4500, 140)

In [76]:
X_train_ls, X_test_ls, y_train_ls, y_test_ls = train_test_split(ecg_wavelet, labels_1, test_size=0.1, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [500, 2627]

In [ ]:
# Define the number of features in the train dataframe
num_features = ecg_data_train.shape[1]
# Reshape the features data to be in the right shape for LSTM input
features = np.asarray(ecg_data_train).astype('float32')
features = features.reshape(features.shape[0], features.shape[1], 1)
X_test_fe = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
# Define the model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(features.shape[1], 1)))
model.add(Dense(1, activation='sigmoid'))
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(features, y_train, validation_data=(X_test_fe, y_test), epochs=20, batch_size=32)
# Make predictions on the validation set


In [ ]:
y_pred = model.predict(X_test_fe)

y_pred = [1 if p>0.5 else 0 for p in y_pred]
print(np.unique(y_pred))

In [ ]:
X_test_final = test_ecg_data.reshape(test_ecg_data.shape[0], test_ecg_data.shape[1], 1)
y_test_pred=model.predict(X_test_final)
y_test_pred = [1 if p>0.5 else 0 for p in y_test_pred]
acc = accuracy_score(test_labels, y_test_pred)
acc

acc


In [ ]:
# calculate the accuracy
acc = accuracy_score(test_labels, y_test_pred)

#classification report provides all metrics e.g. precision, recall, etc.
all_met = classification_report(test_labels, y_test_pred)

In [ ]:

all_met = classification_report(test_labels, y_test_pred)
print("Accuracy: ", acc*100, "%")


print("Classification Report: \n", all_met)


In [ ]:
len(val_losses)